In [3]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np

In [4]:
# MNIST dataset parameters.
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28

# Training parameters.
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

### prepare mnist dataset

In [5]:
from tensorflow.keras.datasets import mnist
(x_train, y_train),(x_test,y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1,num_features]), x_test.reshape([-1,num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train , x_test = x_train / 255. , x_test / 255.


In [6]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-07-25 00:07:11.381248: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-25 00:07:11.381439: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# Weight of shape [784, 10], the 28*28 image features, and total number of classes.
W = tf.Variable(tf.ones([num_features, num_classes]),name='weight')
# Bias of shape [10], the total number of classes.
b = tf.Variable(tf.zeros([num_classes]), name='bias')

# Logistic regression (Wx + b).
def logistic_regression(x):
     # Apply softmax to normalize the logits to a probability distribution. -> 확률 분포 정규화 
    return tf.nn.softmax(tf.matmul(x,W) + b) # 소프트맥스 -> 다중 클래스 분류 (시그모이드 여러개 사용)

# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy. 손실함수
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred),1))

# Accuracy metric. 검증데이터로 정확성 판단
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [8]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    
    gradients = g.gradient(loss,[W,b])
    
    optimizer.apply_gradients(zip(gradients, [W,b]))

In [9]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 1.875846, accuracy: 0.652344
step: 100, loss: 1.594959, accuracy: 0.750000
step: 150, loss: 1.355469, accuracy: 0.769531
step: 200, loss: 1.166040, accuracy: 0.851562
step: 250, loss: 1.113768, accuracy: 0.792969
step: 300, loss: 0.955818, accuracy: 0.835938
step: 350, loss: 0.979679, accuracy: 0.800781
step: 400, loss: 0.869288, accuracy: 0.820312
step: 450, loss: 0.753920, accuracy: 0.890625
step: 500, loss: 0.812929, accuracy: 0.839844
step: 550, loss: 0.799484, accuracy: 0.808594
step: 600, loss: 0.845265, accuracy: 0.816406
step: 650, loss: 0.759374, accuracy: 0.855469
step: 700, loss: 0.668108, accuracy: 0.886719
step: 750, loss: 0.737196, accuracy: 0.804688
step: 800, loss: 0.634252, accuracy: 0.855469
step: 850, loss: 0.745105, accuracy: 0.851562
step: 900, loss: 0.634785, accuracy: 0.843750
step: 950, loss: 0.626457, accuracy: 0.871094
step: 1000, loss: 0.597961, accuracy: 0.859375


In [10]:
pred = logistic_regression(x_test)
print('accuracy: %f' % accuracy(pred, y_test))

accuracy: %f tf.Tensor(0.87039995, shape=(), dtype=float32)
